In [3]:
# Extracción de texto 
data_path = "../data/raw_text/Databricks Certified Associate Developer for Apache Spark.pdf"
pattern = r'(?m)^\s*\d+\s*$' # eliminar líneas con sólo números (numero de página del libro)

import fitz
import re

def delete_lines_w_digits_only(text):
    return re.sub(pattern, '', text) 

def delete_strange_characters(text):
    """Son caracteres que salen de los bloques de código, queremos eliminarlos"""
    return re.sub("\u202f", "", text)

def remove_trailing_section_by_keywords(text, keywords=("Sample questions", "Sample question", "Answer")):
    """
    Elimina el bloque desde que aparece alguna keyword hasta el final del texto.
    Sensible a líneas que comienzan encabezando una sección indeseada.
    """
    lines = text.splitlines()
    lower_keywords = [kw.lower() for kw in keywords]
    
    for i, line in enumerate(lines):
        if line.strip().lower() in lower_keywords:
            return "\n".join(lines[:i]).strip()  # corta desde esa línea hasta el final
    return text  # si no se encuentra, se mantiene todo

## EXTRAER EL TEXTO DE SÓLO LAS PÁGINAS QUE NOS INTERESAN
def extract_range_of_pages(data_path, start_page=31, end_page=208, add_pages_sep=True):
    doc = fitz.open(data_path)
    selected_text = ""

    for n_page in range(start_page, end_page + 1):
        page = doc.load_page(n_page)
        text_from_page = page.get_text()

        # Limpieza: eliminar líneas con solo dígitos
        cleaned_text = delete_lines_w_digits_only(text_from_page)

        # Eliminar contenido desde "Sample questions" o "Answer" 
        cleaned_text = delete_strange_characters(cleaned_text)
        cleaned_text = remove_trailing_section_by_keywords(cleaned_text)
        
        if cleaned_text.strip():  # evitar páginas vacías
            if add_pages_sep:
                selected_text += f"\n--- New page {n_page} ---\n"
            selected_text += cleaned_text + "\n"

    return selected_text, doc

content, doc = extract_range_of_pages(data_path, add_pages_sep=False)

with open("../data/plain_text/plain_text.txt", "w", encoding="utf-8") as out: # fichero de salida limpio
    out.write(content)